In [1]:
import requests, os, re
import gevent
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
def get_selenium_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    options.add_argument("lang=ko_KR")
    fdriver = webdriver.Chrome('../.dev/bin/chromedriver', options=options)

    return fdriver

In [3]:
def get_item_lists(driver):
    while True:
        try:
            btn = driver.find_element_by_css_selector('button.btnMore.js-more')
            btn.click()
        except NoSuchElementException:
            break

    elem = driver.find_element_by_css_selector('ul.imgListWrap')
    html = elem.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    lists = soup.findChildren('li')
    return lists

In [4]:
def make_dirs():
    curdir = os.path.abspath(os.path.curdir)
    _imgdir = os.path.join(curdir, 'site-images')

    if not os.path.isdir(_imgdir):
        os.mkdir(_imgdir)
    return _imgdir

In [5]:
def download_images(img_url, img_path):
    r = requests.get(img_url, stream=True)
    with open(img_path, 'wb') as f:
        f.write(r.content)

In [6]:
url = 'http://www.happymoney.co.kr/svc/store/onlineStore.hm#quick'

driver = get_selenium_driver()
driver.get(url)
item_lists = get_item_lists(driver)
imgdir = make_dirs()

# 이미지 이름 가져오는 re.compile
ignorecase = re.compile(r'\w+/useStore/(.*)', re.IGNORECASE)
# importStore URL 작성을 위해 번호를 가져옴
regex_import_store_number = re.compile(r"'(\w+)'")

# importStore 사이트로 이동하기위해
import_url_front = f"http://www.happymoney.co.kr/svc/store/useStoreView.hm?useStoreInfoId="
import_url_back = "&pageLink=store/onlineStore.hm"

In [7]:
thread_list = []

for item in item_lists:
    img_url = item.select_one('img')['src']
    img_name = ignorecase.search(img_url).group(1)
    img_path = os.path.join(imgdir, img_name)
    
    thread_list.append(gevent.spawn(download_images(img_url, img_path)))

    site_name = item.select_one('div.info > a').get_text()
    category = (item.select_one('div.info > span').get_text() + '/').split('/')[0]
    is_fee = True if item.select_one('strong.ir.ico.feesIcon') else False
    is_import_point = True if item.select_one('strong.ir.ico.storeLab') else False
    month_pay_limit = 0
    # 입점몰의 site_url은 다른 사이트의 iframe을 요구하기때문에 구현하지 않는다.
    site_url = ''

    if is_import_point:
        import_store_number = regex_import_store_number.search(
            item.select_one('div.info > a')['onclick']).group(1)
        url = import_url_front + import_store_number + import_url_back
        driver.get(url)
        month_pay_limit = driver.find_element_by_css_selector('span#paymentMonthLimit').text
        print(month_pay_limit)
        if month_pay_limit == '없음':
            month_pay_limit = 0
        else:
            month_pay_limit = int(''.join(list(filter(str.isdigit, month_pay_limit))))
        
        print(month_pay_limit)
    else:
        site_url = item.select_one('span.btn > a')['href']
        
gevent.joinall(thread_list)

없음
0
없음
0
500,000원
500000
없음
0


KeyboardInterrupt: 

In [ ]:
s = '123abc'
i = int(''.join(list(filter(str.isdigit, s))))

In [ ]:
i